# Извлечение именованных сущностей.

Именованные сущности - общий термин, который используют для обозначения какого-то множества слов (словосочетаний, последовательностей символов), которые представляют какой-то особый инетерес в контексте решаемой практической задачи и которые нужно отделить от остальных слов. Это могут быть какие-то стандартные вещи: имена, фамилии, названия организаций, локации, денежные суммы, даты, номера телефонов, а могут быть и более специфичные: статьи кодексов, белки, симптомы, товары.


В многих случаях достаточно написать регулярку, чтобы решить задачу на приемлемом уровне (например собрать все номера телефонов совсем несложно). Но когда сущности выражются нестандартным образом, а множество представителей сущности постоянно расширяется - задача становится очень сложной.

Стандартные сущности ещё можно извлекать с помощью готовых инструментов.

Для английского удобно использовать [SpaCy](https://spacy.io/). Там сразу извлекаются сущности с хорошим качеством.

Для русского (если не хочется ничего делать) можно использовать тэги из pymorphy.

In [129]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [130]:
p = morph.parse('Михаил')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,masc,Name sing,nomn
True


In [131]:
p = morph.parse('Иванов')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

Тэги -  NOUN,anim,masc,Sgtm,Surn sing,nomn
True


In [132]:
p = morph.parse('Петрович')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Patr sing,nomn
True


In [137]:
p = morph.parse('Одинцово')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,neut,Sgtm,Geox sing,nomn
True


In [138]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Orgn sing,nomn
True


In [142]:
p = morph.parse('МГУ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,gent
True


## &uarr;&uarr;&uarr;
### Попробуйте другие слова для этих типов сущностей, чтобы понять, что работает, а что нет

Рядом стоящие слова одного тэга можно склеить в один. Или сначала собрать нграмы и если какое-то одно слово в нграмме принадлежит к какому-то типу сущности, то распространить его на весь нграм.

Другой инструмент - natasha (https://github.com/natasha/natasha)

Она основана на парсере yargy https://github.com/natasha/yargy и представляет собой набор готовых правил для извлечения некоторых сущностей.

In [143]:
from natasha import (NamesExtractor,
                     SimpleNamesExtractor,
                     PersonExtractor,
                     LocationExtractor,
                     AddressExtractor,
                     OrganisationExtractor,
                     DatesExtractor,
                     MoneyExtractor,
                     MoneyRateExtractor,
                     MoneyRangeExtractor)

from natasha.markup import (show_markup_notebook as show_markup,
                            format_json)

In [146]:
text = 'Влад Веселов. Андрей Петрович. Алиса Викторовна. Студия Артемия Лебедева'

extractor_per = NamesExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [147]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = PersonExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [5]:
text = 'Более того в Москве, в районе Строгино. На реке Оке. В германии'

extractor_loc = LocationExtractor()
matches = extractor_loc(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [157]:
text = 'ФСБ. Московский государственный университет. Высшая школа экономики. ВШЭ. Mail.ru '

extractor_org = OrganisationExtractor()
matches = extractor_org(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [161]:
text = 'С 2015 г. по 2017 год. 16 апреля 1993 года. В четверг . 23.04.18'

extractor_date = DatesExtractor()
matches = extractor_date(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [168]:
text = "Он заплатил ему 3 000 000 000 рублей."

extractor_money = MoneyExtractor()
matches = extractor_money(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

Ещё есть томита-парсер, но с ним очень тяжело работать (никакого развития, скудная документация, закрытый код, никакого сообщества) https://tech.yandex.ru/tomita/

Если нужно улучшить выделение стандартныйх типов или научиться извлекать специфичные сущности, нужно писать правила в Yargy. 

Возьмем тексты объявлений на авито по категории Консоли и попробуем научиться выделять названия приставок Xbox и Playstation.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

Посмотрим на данные.

In [2]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [4]:
data.shape

(9285, 1)

Давайте попробуем сначала полные варианты: Xbox 360, Xbox one, Playstation 1,2,3,4.

In [171]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

Для начала определим саму сущность, которую будет извлекать. Сущность будет называться Pristavka и неё будет два атрибута: название и версия. Делается это вот так:

In [175]:

Pristavka = fact(
    'Pristavka',
    ['name', 'model']
)

(Чуть позже станет понятнее зачем так делать)

Теперь сделаем правило для иксбокса. Просто напишем несколько вариантов написания Xbox и версий.
Такой список называется Газзетир. В Yargy его удобно задавать через morph_pipeline. Туда можно написать слова или последовательности, а он их нормализует, чтобы потом сопоставлять с текстом.


В итоге у нас два газзетира. Мы оборачиваем их в правило (rule). Работать это будет примерно как регулярка. 

In [176]:
Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name), # то, что сматчиться будет в атрибуте name
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model)) # то, что сматчиться будет в атрибуте version


Правило для плейстешена будет аналогичное.

In [177]:
PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model)    
    )

Теперь сделаем общее правило, которое будет искать плейстешены или иксбоксы и извлекать их как сущность типа Pristavka.

In [178]:
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

Попробуем сматчить по текстам.

In [179]:
matches = []

for sent in data.text[:100]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [180]:
for m in matches:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2
Xbox 360
PS 4
PS 3
PS 4
PS 3
Xbox 360
Xbox One
PS 4
Xbox 360
Xbox One
PlayStation 3
xbox 360
PS 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox One
ps 4
ps 4
Xbox 360
PS 3
PS 3
XBox 360
XBox 360
Ps 3


У приставок ещё могут быть подификации вроде Slim, X, S и т.д, но не всегда. Чтобы учесть это можно добавить ещё одно правило с .optional() на конце.

In [184]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [185]:
matches = []

for sent in data.text[:1000]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [188]:
list(parser.findall('Playstation 3 fdf'))[0].fact

Pristavka(name='Playstation',
          model='3',
          version=None)

In [186]:
for m in matches:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2
Xbox 360
PS 4
PS 3
PS 4
PS 3
Xbox 360
Xbox One
PS 4
Xbox 360
Xbox One
PlayStation 3
xbox 360
PS 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox One
ps 4
ps 4
Xbox 360
PS 3
PS 3
XBox 360
XBox 360
Ps 3
PS 3
PS 3
PS 4
ps 4
PS 4
xbox 360
X box One
xbox 360
PS 3
XBOX 360
XBOX 360
Playstation 3
PS 2
PlayStation 3
PlayStation 3
PlayStation 3
Playstation 3
Playstation 4
Xbox 360
Xbox One
XBOX ONE
PS 2
PS 4
Xbox One
PS 4
PS 4
playstation 3
PS 3
Xbox one
PS 4
ps 2
ps 3
PS 4
Xbox 360
PS 4
PlayStation 4
PS 4
PS 4
PlayStation 4
PlayStation 4
PlayStation 4
PS 4
Playstation 4
PS 4
Playstation 4
PS 4
Playstation 4
PS 3
Xbox 360
playstation 4
Xbox 360
ps 3
Xbox 360
Playstation 4
Ps 4
ps 4
ps 4
xbox one
PS 4
PS 2
Xbox One
Playstation 2
PS 4
PS 4
xbox one
PS 4
PS 4
xbox ONE
PS 3
Playstation 3
Playstation 3
xbox 360
PS 4
playstation 4
XBOX 360
PlayStation 3
Xbox One
Play Station 2
ps 3
PS 3
PS 4
PS 3
XBOX 360
PS 2
ps 3
PS 3
PS 4


Все работает как и должно.

Хорошо бы ещё приводить все варианты названия к нормальному виду. Вот как это можно сделать.

Если можно заменить весь газзетир на какое-то одно слово, то можно просто добавить в интерпретации значение .сonst('something')

In [189]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [190]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [191]:
for m in matches:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox One
Playstation 4
Xbox 360
Xbox One
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox One
Playstation 4
Playstation 4
Xbox 360
Playstation 3
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox One
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Xbox 360
Xbox One
Xbox ONE
Playstation 2
Playstation 4
Xbox One
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4


В общем случае к нормальной форме приводит .normalized() добавленный к атрибуту в интерпретации. Но он работает через pymorphy2, а он работает с русским языком. Поэтому в нашем случае его можно использовать только для привода к нижнему регистру.

Когда у каждого и слов в газзетире есть своя нормальная форма, можно сделать газзетир словарём, где ключи это нужные формы, а значения - нормальные формы. А в интерпретации к .normalized() добавить .custom() и через него дергать нужную правильную форму.

In [192]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model.normalized()),
    in_caseless('sxe').interpretation(Pristavka.version.normalized()).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    # для простоты можно написать вот так
    in_('1234').interpretation(Pristavka.model.normalized()),
    #изменения вот тут                                                      вот тут дергаем правильную форму
    caseless_pipeline(VERSIONS).interpretation(Pristavka.version.normalized().custom(VERSIONS.get)).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [195]:
list(parser.findall('PS 3 SlIm'))[0].fact

Pristavka(name='Playstation',
          model='3',
          version='Slim')

In [125]:
list(parser.findall('XbOx 360 X'))[0].fact

Pristavka(name='Xbox',
          model='360',
          version='x')

In [126]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [127]:
for m in matches:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox one
Playstation 4
Xbox 360
Xbox one
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox one
Playstation 4
Playstation 4
Xbox 360
Playstation 3
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox one
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Xbox 360
Xbox one
Xbox one
Playstation 2
Playstation 4
Xbox one
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4


Улучшения для этого типа сущностей довольно очевидные и простые - нужно расширять газзетиры, можно добавить другие приставки вроде денди и нинтендо.

Для сущностей, которые нельзя перечислить словарём, нужны контекстные правила. В целом они не сильно сложнее того, что мы сейчас написали. Возможно вам понадобятся другие полезные штуки из Yargy (предикат - and_, учёт грамматической информации - gram('NOUN'), согласование по роду, числу и падежу - gnc).

Учить сразу все не имеет смысла. Возьмите практическую задачу, документацию к Yargy (https://yargy.readthedocs.io/ru/latest/index.html) и разбирайте только то, что вам нужно. 